In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, PrecisionRecallDisplay, RocCurveDisplay
from sklearn.metrics import f1_score, roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay



In [13]:

# Loading data
customers = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
transactions = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')
engagement = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')

# Initial exploration
print(customers.head())
print(transactions.head())
print(engagement.head())
print(marketing.head())



   customer_id   join_date last_purchase_date   age  gender           location
0            1  2023-11-20         2024-03-17  56.0  Female  North Shannonbury
1            2  2021-09-08         2023-10-25   NaN    Male          Hillville
2            3  2021-06-01         2022-11-27   NaN     NaN   North Latoyatown
3            4  2022-01-01         2022-09-01  29.0    Male          Grossstad
4            5  2022-01-24         2023-06-02   NaN    Male   East Matthewfort
   transaction_id  customer_id transaction_date  transaction_amount  \
0               1            1       2024-02-03              165.56   
1               2            1       2024-03-02              699.01   
2               3            1       2024-03-12              146.86   
3               4            1       2024-01-20              927.46   
4               5            1       2024-02-25             1395.87   

  product_category  
0         Clothing  
1       Home Goods  
2       Home Goods  
3      Electron

In [46]:

# Loading data from URLs
customers = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
transactions = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')
engagement = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')

# Merge datasets
merged_df = transactions.merge(customers, on='customer_id', how='left')
merged_df = merged_df.merge(engagement, on='customer_id', how='left')
merged_df = merged_df.merge(marketing, on='customer_id', how='left')

# Define the cutoff date
cutoff_date = pd.to_datetime('2024-05-31')

In [47]:
# Convert date columns to datetime format
merged_df['join_date'] = pd.to_datetime(merged_df['join_date'])
merged_df['last_purchase_date'] = pd.to_datetime(merged_df['last_purchase_date'])
merged_df['campaign_date'] = pd.to_datetime(merged_df['campaign_date'])
merged_df['transaction_date'] = pd.to_datetime(merged_df['transaction_date'])

# Define the cutoff date
cutoff_date = pd.to_datetime('2024-05-31')

# Convert dates later than 5/31/2024 to 5/31/2024
merged_df['join_date'] = merged_df['join_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)
merged_df['last_purchase_date'] = merged_df['last_purchase_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)
merged_df['campaign_date'] = merged_df['campaign_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)
merged_df['transaction_date'] = merged_df['transaction_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)

# Display the updated DataFrame to verify
print(merged_df.head())

   transaction_id  customer_id transaction_date  transaction_amount  \
0               1            1       2024-02-03              165.56   
1               1            1       2024-02-03              165.56   
2               1            1       2024-02-03              165.56   
3               1            1       2024-02-03              165.56   
4               2            1       2024-03-02              699.01   

  product_category  join_date last_purchase_date   age  gender  \
0         Clothing 2023-11-20         2024-03-17  56.0  Female   
1         Clothing 2023-11-20         2024-03-17  56.0  Female   
2         Clothing 2023-11-20         2024-03-17  56.0  Female   
3         Clothing 2023-11-20         2024-03-17  56.0  Female   
4       Home Goods 2023-11-20         2024-03-17  56.0  Female   

            location  number_of_site_visits  number_of_emails_opened  \
0  North Shannonbury                     10                       15   
1  North Shannonbury            

In [48]:
# verifying names
print(merged_df.columns)


Index(['transaction_id', 'customer_id', 'transaction_date',
       'transaction_amount', 'product_category', 'join_date',
       'last_purchase_date', 'age', 'gender', 'location',
       'number_of_site_visits', 'number_of_emails_opened', 'number_of_clicks',
       'campaign_id', 'response', 'promotion_type', 'campaign_date'],
      dtype='object')


Data Cleaning and Preparation

In [49]:
# Handling missing values in customers dataset
customers['age'].fillna(customers['age'].median(), inplace=True)
customers['gender'].fillna('Unknown', inplace=True)

# Handling missing values in transactions dataset (now 'merged_df')
merged_df['transaction_amount'].fillna(0, inplace=True)  # Adjust based on actual column names
merged_df['number_of_site_visits'].fillna(0, inplace=True)  # Adjust based on actual column names
merged_df['number_of_clicks'].fillna(0, inplace=True)  # Adjust based on actual column names
merged_df['number_of_emails_opened'].fillna(0, inplace=True)  # Adjust based on actual column names



C:\Users\aayre\AppData\Local\Temp\ipykernel_31996\2648064231.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers['age'].fillna(customers['age'].median(), inplace=True)
C:\Users\aayre\AppData\Local\Temp\ipykernel_31996\2648064231.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [50]:
# Convert date columns to datetime format
merged_df['transaction_date'] = pd.to_datetime(merged_df['transaction_date'])
merged_df['last_purchase_date'] = pd.to_datetime(merged_df['last_purchase_date'])
merged_df['campaign_date'] = pd.to_datetime(merged_df['campaign_date'])



In [51]:
# Print the columns of merged_df to verify the names
print(merged_df.columns)


Index(['transaction_id', 'customer_id', 'transaction_date',
       'transaction_amount', 'product_category', 'join_date',
       'last_purchase_date', 'age', 'gender', 'location',
       'number_of_site_visits', 'number_of_emails_opened', 'number_of_clicks',
       'campaign_id', 'response', 'promotion_type', 'campaign_date'],
      dtype='object')


Feature Engineering

In [52]:
# Define the cutoff date
cutoff_date = pd.Timestamp('2023-07-01')

# Feature engineering on merged_df
merged_df['recency'] = (cutoff_date - merged_df['last_purchase_date']).dt.days
merged_df['frequency'] = merged_df.groupby('customer_id')['transaction_id'].transform('count')
merged_df['monetary'] = merged_df.groupby('customer_id')['transaction_amount'].transform('sum')
merged_df['response_rate'] = merged_df['response'].apply(lambda response: 1 if response == 'Yes' else 0)
merged_df['clicks_per_transaction'] = merged_df['number_of_clicks'] / merged_df['frequency']

# Handle NaN or infinite values
merged_df['clicks_per_transaction'].fillna(0, inplace=True)

# Display the updated DataFrame to verify
print(merged_df.head())


   transaction_id  customer_id transaction_date  transaction_amount  \
0               1            1       2024-02-03              165.56   
1               1            1       2024-02-03              165.56   
2               1            1       2024-02-03              165.56   
3               1            1       2024-02-03              165.56   
4               2            1       2024-03-02              699.01   

  product_category  join_date last_purchase_date   age  gender  \
0         Clothing 2023-11-20         2024-03-17  56.0  Female   
1         Clothing 2023-11-20         2024-03-17  56.0  Female   
2         Clothing 2023-11-20         2024-03-17  56.0  Female   
3         Clothing 2023-11-20         2024-03-17  56.0  Female   
4       Home Goods 2023-11-20         2024-03-17  56.0  Female   

            location  ...  number_of_clicks  campaign_id  response  \
0  North Shannonbury  ...                 1            1        No   
1  North Shannonbury  ...           

C:\Users\aayre\AppData\Local\Temp\ipykernel_31996\4173890770.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['clicks_per_transaction'].fillna(0, inplace=True)


In [53]:
# adjusting dates handling

# Define the cutoff date
cutoff_date = pd.Timestamp('2024-05-31')

# Convert dates later than the cutoff date to the cutoff date itself
merged_df['join_date'] = merged_df['join_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)
merged_df['last_purchase_date'] = merged_df['last_purchase_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)
merged_df['campaign_date'] = merged_df['campaign_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)
merged_df['transaction_date'] = merged_df['transaction_date'].apply(lambda x: x if x <= cutoff_date else cutoff_date)


In [54]:
# feature engineering with adjustments

# Feature engineering on merged_df
merged_df['recency'] = (cutoff_date - merged_df['last_purchase_date']).dt.days
merged_df['frequency'] = merged_df.groupby('customer_id')['transaction_id'].transform('count')
merged_df['monetary'] = merged_df.groupby('customer_id')['transaction_amount'].transform('sum')
merged_df['response_rate'] = merged_df['response'].apply(lambda response: 1 if response == 'Yes' else 0)
merged_df['clicks_per_transaction'] = merged_df['number_of_clicks'] / merged_df['frequency']

# Handle NaN or infinite values
merged_df['clicks_per_transaction'].fillna(0, inplace=True)

# Display the updated DataFrame to verify
print(merged_df.head())


   transaction_id  customer_id transaction_date  transaction_amount  \
0               1            1       2024-02-03              165.56   
1               1            1       2024-02-03              165.56   
2               1            1       2024-02-03              165.56   
3               1            1       2024-02-03              165.56   
4               2            1       2024-03-02              699.01   

  product_category  join_date last_purchase_date   age  gender  \
0         Clothing 2023-11-20         2024-03-17  56.0  Female   
1         Clothing 2023-11-20         2024-03-17  56.0  Female   
2         Clothing 2023-11-20         2024-03-17  56.0  Female   
3         Clothing 2023-11-20         2024-03-17  56.0  Female   
4       Home Goods 2023-11-20         2024-03-17  56.0  Female   

            location  ...  number_of_clicks  campaign_id  response  \
0  North Shannonbury  ...                 1            1        No   
1  North Shannonbury  ...           

C:\Users\aayre\AppData\Local\Temp\ipykernel_31996\1095616153.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['clicks_per_transaction'].fillna(0, inplace=True)


In [91]:
len(merged_df)

327251

Model Buildign and Evaluation

In [55]:
# prep initial
# Define features (X) and target (y)
features = ['age', 'gender', 'location', 'number_of_site_visits', 'number_of_emails_opened',
            'number_of_clicks', 'campaign_id', 'response', 'promotion_type', 'campaign_date',
            'recency', 'frequency', 'response_rate', 'clicks_per_transaction']

X = merged_df[features]
y = merged_df['response']  # Assuming 'response' is the target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Model Selection and Evaluation

In [62]:
#Preprocessing Pipeline

# Importing necessary libraries
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


# Define preprocessing steps
numeric_features = ['age', 'number_of_site_visits', 'number_of_emails_opened', 'number_of_clicks',
                    'campaign_id', 'recency', 'frequency', 'response_rate', 'clicks_per_transaction']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())  # Scale numeric features
])

categorical_features = ['gender', 'location', 'promotion_type']

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Encode categorical features
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])



In [77]:
# Check if all expected columns are present
expected_columns = [
    'transaction_id', 'customer_id', 'transaction_date', 'transaction_amount', 'product_category',
    'join_date', 'last_purchase_date', 'age', 'gender', 'location', 'number_of_site_visits',
    'number_of_emails_opened', 'number_of_clicks', 'campaign_id', 'response', 'promotion_type',
    'campaign_date', 'recency', 'frequency', 'monetary', 'response_rate', 'clicks_per_transaction'
]

if not all(col in merged_df.columns for col in expected_columns):
    missing_cols = [col for col in expected_columns if col not in merged_df.columns]
    raise ValueError(f"Missing columns in DataFrame: {missing_cols}")


In [78]:
print(merged_df.columns) 

Index(['transaction_id', 'customer_id', 'transaction_date',
       'transaction_amount', 'product_category', 'join_date',
       'last_purchase_date', 'age', 'gender', 'location',
       'number_of_site_visits', 'number_of_emails_opened', 'number_of_clicks',
       'campaign_id', 'response', 'promotion_type', 'campaign_date', 'recency',
       'frequency', 'monetary', 'response_rate', 'clicks_per_transaction'],
      dtype='object')


In [90]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Make sure to list your actual dataset columns here
columns = list(X_train.columns)

# Defining the preprocessor
numeric_features = ['recency', 'frequency', 'monetary']
categorical_features = ['gender', 'location', 'promotion_type']

# Verify if all the features are in the columns
for feature_set in [numeric_features, categorical_features]:
    for feature in feature_set:
        if feature not in columns:
            print(f"Warning: {feature} is not found in the dataset columns!")

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the models
models = {
    'RandomForestRegressor': RandomForestRegressor(),
    'LinearRegression': LinearRegression(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'LogisticRegression': LogisticRegression(),
    'GaussianNB': GaussianNB(),
    'RandomForestClassifier': RandomForestClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier()
}

# Evaluate each model
for model_name, model in models.items():
    print(f"Evaluating model: {model_name}")
    try:
        scores = cross_val_score(
            Pipeline(steps=[('preprocessor', preprocessor), ('model', model)]), 
            X_train, y_train, cv=5, 
            scoring='accuracy' if isinstance(model, (LogisticRegression, GaussianNB, KNeighborsClassifier, RandomForestClassifier)) else 'neg_mean_squared_error'
        )
        if isinstance(model, (LogisticRegression, GaussianNB, KNeighborsClassifier, RandomForestClassifier)):
            print(f"Cross-validation accuracy for {model_name}: {scores.mean():.2f} (+/- {scores.std() * 2:.2f})")
        else:
            print(f"Cross-validation MSE for {model_name}: {-scores.mean():.2f} (+/- {scores.std() * 2:.2f})")
    except Exception as e:
        print(f"Failed to evaluate model {model_name}: {e}")




Evaluating model: RandomForestRegressor
Failed to evaluate model RandomForestRegressor: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\aayre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\\_libs\\has

In [82]:
# Labeling VIFs 

# Assuming you have calculated VIF scores
vif_scores = {
    'age': 5.2, 
    'number_of_site_visits': 2.1,
    # Add more VIF scores as per your analysis
}

vif_labels = {}

for feature, vif in vif_scores.items():
    if vif > 10:
        vif_labels[feature] = 'high-value'
    else:
        vif_labels[feature] = 'low-level'

print("VIF Labels:")
for feature, label in vif_labels.items():
    print(f"{feature}: {label}")
